# WRFcast Tutorial

This tutorial will walk through forecast data from your own WRF forecast model data using the wrfcast.py module within pvlib.

This tutorial has been tested against the following package versions:
* Python 
* IPython 
* pandas 
* matplotlib 
* netcdf4 1.4.2 

It should work with other Python and Pandas versions. It requires pvlib >= 0.3.0 and IPython >= 3.0.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, November 2019

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect

# python add-ons
import numpy as np
import pandas as pd
import netCDF4

import pvlib
from pvlib.wrfcast import WRF

/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF Solar and Wind/pvlib-python/pvlib/wrfcast.py:20: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  'The forecast module algorithms and features are highly experimental. '


In [2]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

In [3]:
data = netCDF4.Dataset(datapath)

In [4]:
dims = data.dimensions
ndims = len(dims)
print(f'This wrfout file has {ndims} dimensions.')

This wrfout file has 9 dimensions.


In [5]:
# Now print the length of each dimension.
for key in dims:
    print(f'dimension[{key}] = {len(dims[key])}')

dimension[Time] = 24
dimension[DateStrLen] = 19
dimension[west_east] = 191
dimension[south_north] = 191
dimension[bottom_top] = 35
dimension[bottom_top_stag] = 36
dimension[soil_layers_stag] = 4
dimension[west_east_stag] = 192
dimension[south_north_stag] = 192


In [6]:
gattrs = data.ncattrs() # Forms a dict of attriburtes
ngattrs = len(gattrs)
print(f'There are {ngattrs} global attributes.')

There are 126 global attributes.


In [7]:
# Now print the global attributes.
for key in gattrs:
    print(f'Global attribute[{key}] = {getattr(data, key)}')

Global attribute[TITLE] =  OUTPUT FROM WRF V3.8.1 MODEL
Global attribute[START_DATE] = 2011-01-24_00:00:00
Global attribute[SIMULATION_START_DATE] = 2011-01-23_12:00:00
Global attribute[WEST-EAST_GRID_DIMENSION] = 192
Global attribute[SOUTH-NORTH_GRID_DIMENSION] = 192
Global attribute[BOTTOM-TOP_GRID_DIMENSION] = 36
Global attribute[DX] = 12000.0
Global attribute[DY] = 12000.0
Global attribute[SKEBS_ON] = 0
Global attribute[SPEC_BDY_FINAL_MU] = 1
Global attribute[USE_Q_DIABATIC] = 0
Global attribute[GRIDTYPE] = C
Global attribute[DIFF_OPT] = 1
Global attribute[KM_OPT] = 4
Global attribute[DAMP_OPT] = 0
Global attribute[DAMPCOEF] = 0.20000000298023224
Global attribute[KHDIF] = 0.0
Global attribute[KVDIF] = 0.0
Global attribute[MP_PHYSICS] = 10
Global attribute[RA_LW_PHYSICS] = 1
Global attribute[RA_SW_PHYSICS] = 1
Global attribute[SF_SFCLAY_PHYSICS] = 2
Global attribute[SF_SURFACE_PHYSICS] = 2
Global attribute[BL_PBL_PHYSICS] = 5
Global attribute[CU_PHYSICS] = 3
Global attribute[SF_LAKE

In [8]:
vars = data.variables # Loads variables into a dict
nvars = len(vars)
print(f'There are {nvars} variables.')

There are 156 variables.


In [27]:
# Now for each variable, query its dimensions and attributes
# variables = {
#             'temp_air': 'T2', # TEMP at 2 M
#             'wind_speed_u': 'U10', # U at 10 M
#             'wind_speed_v': 'V10', # V at 10 M
#             'total_clouds': 'CLDFRA', #  CLOUD FRACTION
#             'dni': 'Shortwave surface downward direct normal irradiance', # SWDDNI
#             'dhi': 'Shortwave surface downward diffuse irradiance' # SWDDIF
#             }
variables = [
            'T2', # T2
            'U10',
            'V10',
            'CLDFRA',
            'SWDDNI',
            'SWDDIF'
            ]
for var in variables:
    print(f'- - - - - Variable: {var} - - - - -')
    print(f'Shape = {vars[var].shape}') # Shape is a tuple
    vdims = vars[var].dimensions
    for vd in vdims:
        print(f'dimension[{vd}] = {len(dims[vd])}') # Print the length
    # Now for each variable, let's query its dimensions and attributes for var in vars
    vattrs = vars[var].ncattrs() # Vattars is a dict
    print(f'Number of attributes = {len(vattrs)}')
    for vat in vattrs:
        print(f'attribute[{vat}] = {getattr(vars[var], vat)}')
    # Now just get a slice of data
    a = vars[var][1:3]
    print(a)

- - - - - Variable: T2 - - - - -
Shape = (24, 191, 191)
dimension[Time] = 24
dimension[south_north] = 191
dimension[west_east] = 191
Number of attributes = 6
attribute[FieldType] = 104
attribute[MemoryOrder] = XY 
attribute[description] = TEMP at 2 M
attribute[units] = K
attribute[stagger] = 
attribute[coordinates] = XLONG XLAT XTIME
[[[287.05905 287.0014  287.1845  ... 290.3643  290.39136 290.2971 ]
  [287.0477  287.00604 286.96536 ... 289.9462  289.9016  290.19135]
  [286.94843 286.93137 286.82693 ... 289.76865 289.83832 290.2409 ]
  ...
  [259.48553 259.016   258.6835  ... 262.00626 262.36765 263.43198]
  [259.393   258.9204  258.57104 ... 262.21954 262.5636  263.49747]
  [259.22183 258.95612 258.5753  ... 262.94476 263.36563 263.67072]]

 [[286.7828  286.74426 286.74155 ... 290.44284 290.4597  290.2889 ]
  [286.47787 286.4802  286.57605 ... 290.0115  289.87888 290.13202]
  [286.37677 286.54477 286.4714  ... 289.75558 289.8061  290.17435]
  ...
  [259.6599  259.24295 258.96667 ... 2